In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

Cargamos todas las librerias que utilizaremos en este notebook, en este caso hay más por que vamos a realizar el notebook 03 hasta la parte de modelo, sin incluir esta última.

In [2]:
pd_loan_train = pd.read_csv("../data/train_pd_data_preprocessing_missing_outlier.csv")
pd_loan_test = pd.read_csv("../data/test_pd_data_preprocessing_missing_outlier.csv")

Leemos los .csv generados al final del notebook2, como siempre en una ruta relativa para que cualquiera que descargue el .zip pueda utilizar el notebook desde su dispositivo

In [3]:
pd_loan_train = pd_loan_train.drop(columns =['Unnamed: 0'])
pd_loan_test = pd_loan_test.drop(columns =['Unnamed: 0'])

Creamos los dataset de train y test, que eran con los que estabamos trabajando al final del notebook 02, eliminando la columna 'Unnamed: 0' generada al exportar los dataframes como .csv al final del notebook 02.

Reviso que en el dataframe solo tenemos las variables con las que hemos estado trabajando, en este caso coincide con las variables continuas ('float64','int64'), booleanas ('0') y categóricas ('object')

In [4]:
pd_loan_train.dtypes.unique()

array([dtype('float64'), dtype('O'), dtype('int64')], dtype=object)

Encontramos como hay variables continuas ('float64', 'int64'), no booleanas ('0') y categóricas ('object')

In [5]:
pd_loan_train['TARGET'].value_counts()

TARGET
0    226148
1     19860
Name: count, dtype: int64

Vemos la distribución que sigue el dataframe de train y quitamos la variable de estudio TARGET de la los X correspondientes a train y test ya que será la variable y como se ve a continuación:

In [6]:
X_train = pd_loan_train.drop('TARGET',axis=1)
X_test = pd_loan_test.drop('TARGET',axis=1)
y_train = pd_loan_train['TARGET']
y_test = pd_loan_test['TARGET']

A continuación, muestro, dentro de las variables categóricas, cuantos diferentes valores tiene cada variable para ver si es viable el 'OneHotEnconding' o si hay algunas a las que aplicar otro tipo de categorías

In [7]:
categorical_counts = pd_loan_train.select_dtypes(include='object').nunique()
categorical_counts_sorted = categorical_counts.sort_values(ascending=False)
print(categorical_counts_sorted)

ORGANIZATION_TYPE             58
OCCUPATION_TYPE               19
WALLSMATERIAL_MODE             8
NAME_TYPE_SUITE                8
NAME_INCOME_TYPE               8
WEEKDAY_APPR_PROCESS_START     7
NAME_HOUSING_TYPE              6
NAME_FAMILY_STATUS             6
FONDKAPREMONT_MODE             5
NAME_EDUCATION_TYPE            5
HOUSETYPE_MODE                 4
EMERGENCYSTATE_MODE            3
CODE_GENDER                    3
NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
dtype: int64


Viendo el número de valores que tiene cada variable, he decidido utilizar para todas las variables, salvo 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE', el OneHotEncoding y para las dos restantes el TargetEncoding.

In [8]:
# Definir la columna objetivo
y_train = pd_loan_train['TARGET'] # O la columna que estés usando como target
y_test = pd_loan_test['TARGET']

# Eliminar la columna objetivo de X_train (las características)
X_train = pd_loan_train.drop(columns=['TARGET'])
X_test = pd_loan_test.drop(columns=['TARGET'])

# Lista de las columnas categóricas, excluyendo 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
list_columns_cat = list(X_train.select_dtypes(include=["object", "category"]).columns)
list_columns_cat = [col for col in list_columns_cat if col not in ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']]

# Crear el codificador OneHotEncoder para las columnas categóricas
ohe = ce.OneHotEncoder(cols=list_columns_cat)

# Crear el codificador TargetEncoder para 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
te = ce.TargetEncoder(cols=['ORGANIZATION_TYPE', 'OCCUPATION_TYPE'])

# Crear un Pipeline que aplique ambos codificadores en secuencia
model = Pipeline(steps=[
    ('ohe', ohe),  # OneHotEncoder para las columnas categóricas
    ('te', te)     # TargetEncoder para 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
])

# Ajustar y transformar los datos
X_train_final = model.fit_transform(X_train, y_train)

# Definir la columna objetivo
y_train = pd_loan_train['TARGET']  # O la columna que estés usando como target

# Eliminar la columna objetivo de X_train (las características)
X_train = pd_loan_train.drop(columns=['TARGET'])

# Lista de las columnas categóricas, excluyendo 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
list_columns_cat = list(X_train.select_dtypes(include=["object", "category"]).columns)
list_columns_cat = [col for col in list_columns_cat if col not in ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']]

# Crear el codificador OneHotEncoder para las columnas categóricas
ohe = ce.OneHotEncoder(cols=list_columns_cat)

# Crear el codificador TargetEncoder para 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
te = ce.TargetEncoder(cols=['ORGANIZATION_TYPE', 'OCCUPATION_TYPE'])

# Crear un Pipeline que aplique ambos codificadores en secuencia
model = Pipeline(steps=[
    ('ohe', ohe),  # OneHotEncoder para las columnas categóricas
    ('te', te)     # TargetEncoder para 'ORGANIZATION_TYPE' y 'OCCUPATION_TYPE'
])

In [12]:
# Ajustar y transformar los datos
X_train_final = model.fit_transform(X_train, y_train) #Ajustamos train al modelo
X_test_final = model.transform(X_test) #Ajustamos el test a través del train

Aplicamos la codificación a train y luego aplicamos la codificación realizada en train a test

Verificamos que se han creado las columnas a través de la codificación, las nuevas columnas, para comprobarlo a la izq del todo, son aquellas como, p.e., NAME_INCOME_1/2/3/.../8

In [13]:
print(len(X_train_final.columns))
print(len(X_test_final.columns))

176
176


El dataset final consta de 176 columnas, se han generado 54 adicionales tras la codificación de variables.

In [14]:
X_train_final.dtypes.to_dict()

{'COMMONAREA_MEDI': dtype('float64'),
 'COMMONAREA_AVG': dtype('float64'),
 'COMMONAREA_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_AVG': dtype('float64'),
 'NONLIVINGAPARTMENTS_MEDI': dtype('float64'),
 'FONDKAPREMONT_MODE_1': dtype('int64'),
 'FONDKAPREMONT_MODE_2': dtype('int64'),
 'FONDKAPREMONT_MODE_3': dtype('int64'),
 'FONDKAPREMONT_MODE_4': dtype('int64'),
 'FONDKAPREMONT_MODE_5': dtype('int64'),
 'LIVINGAPARTMENTS_MODE': dtype('float64'),
 'LIVINGAPARTMENTS_AVG': dtype('float64'),
 'LIVINGAPARTMENTS_MEDI': dtype('float64'),
 'FLOORSMIN_AVG': dtype('float64'),
 'FLOORSMIN_MODE': dtype('float64'),
 'FLOORSMIN_MEDI': dtype('float64'),
 'YEARS_BUILD_MEDI': dtype('float64'),
 'YEARS_BUILD_MODE': dtype('float64'),
 'YEARS_BUILD_AVG': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'LANDAREA_MEDI': dtype('float64'),
 'LANDAREA_MODE': dtype('float64'),
 'LANDAREA_AVG': dtype('float64'),
 'BASEMENTAREA_MEDI': dtype('float64'),
 'BA

Se va a realizar el escalado de variables de ambos dataframes, se utiliza el método estandar de la librería sklearn y el modelo escalado se entrena con X_train_final. Una vez realizado, obtenemos dos dataframes finales con el escalado de variables realizado siendo estos X_train_scaled y X_test_scaled.

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_final)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_final), columns=X_train_final.columns, index=X_train_final.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_final), columns=X_test_final.columns, index=X_test_final.index)

In [16]:
X_train_scaled.describe()

,COMMONAREA_MEDI,COMMONAREA_AVG,COMMONAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE_1,FONDKAPREMONT_MODE_2,FONDKAPREMONT_MODE_3,FONDKAPREMONT_MODE_4,...,NAME_EDUCATION_TYPE_5,NAME_INCOME_TYPE_1,NAME_INCOME_TYPE_2,NAME_INCOME_TYPE_3,NAME_INCOME_TYPE_4,NAME_INCOME_TYPE_5,NAME_INCOME_TYPE_6,NAME_INCOME_TYPE_7,NAME_INCOME_TYPE_8,SK_ID_CURR
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,9.327736e-17,2.849299e-17,8.607108e-17,5.157766e-17,-1.774494e-17,1.656796e-17,9.799611e-17,3.309982e-17,1.623220e-17,5.320232e-17,...,1.256407e-17,-4.200297e-17,1.339517e-16,-7.360810e-17,-5.050899e-18,-1.938765e-18,-4.456092e-18,1.034369e-18,4.897459e-18,1.410786e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-6.517368e-01,-6.567699e-01,-6.211480e-01,-9.440585e-02,-9.963895e-02,-9.846608e-02,-1.469247e+00,-2.021899e-01,-5.632669e-01,-1.365179e-01,...,-2.245669e-02,-5.518975e-01,-1.031755e+00,-4.684926e-01,-2.759282e-01,-8.554163e-03,-6.687002e-03,-4.032356e-03,-5.702659e-03,-1.737226e+00
25%,-1.659417e-01,-1.651623e-01,-1.679240e-01,-9.440585e-02,-9.963895e-02,-9.846608e-02,-1.469247e+00,-2.021899e-01,-5.632669e-01,-1.365179e-01,...,-2.245669e-02,-5.518975e-01,-1.031755e+00,-4.684926e-01,-2.759282e-01,-8.554163e-03,-6.687002e-03,-4.032356e-03,-5.702659e-03,-8.651578e-01
50%,-1.659417e-01,-1.651623e-01,-1.679240e-01,-9.440585e-02,-9.963895e-02,-9.846608e-02,6.806208e-01,-2.021899e-01,-5.632669e-01,-1.365179e-01,...,-2.245669e-02,-5.518975e-01,9.692224e-01,-4.684926e-01,-2.759282e-01,-8.554163e-03,-6.687002e-03,-4.032356e-03,-5.702659e-03,8.555294e-04
75%,-1.659417e-01,-1.651623e-01,-1.679240e-01,-9.440585e-02,-9.963895e-02,-9.846608e-02,6.806208e-01,-2.021899e-01,-5.632669e-01,-1.365179e-01,...,-2.245669e-02,-5.518975e-01,9.692224e-01,-4.684926e-01,-2.759282e-01,-8.554163e-03,-6.687002e-03,-4.032356e-03,-5.702659e-03,8.658492e-01
max,2.270379e+01,2.275311e+01,2.323275e+01,3.770117e+01,3.650255e+01,3.668816e+01,6.806208e-01,4.945846e+00,1.775357e+00,7.325046e+00,...,4.453016e+01,1.811931e+00,9.692224e-01,2.134505e+00,3.624132e+00,1.169021e+02,1.495439e+02,2.479940e+02,1.753568e+02,1.730887e+00


El escalado de variables es interesante realizarlo cuando tenemos variables en escalas muy diferentes y el no realizar el escalado podría hacer que el modelo este sesgado por aquellas variables que tienen valores más grandes. Como se ha mencionado, se ha realizado el escalado estandar, cuya formula es:

$$
z = \frac{x - \mu}{\sigma}
$$

Se ha realizado este escalado por que al restarle la media (mu), los valores quedan muy cercanos a 0, esto es importante para evitar sesgos. También al dividir entre sigma lo que estoy haciendo es ajustar la dispersión de los datos, haciendo que todas las variables tengan la misma magnitud relativa. Por último puede ser interesante para la interpretación de los datos ya que como he mencionado anteriormente este método elimina las escalas y se facilita la interpretación de las contribuciones relativas de cada variable.


CONCLUSIONES

Por último tenemos las conclusiones que saco, teniendo en cuenta que aún no se ha realizado el modelo.
Las principales conclusiones son que hay ciertas variables que si parecen afectar a la hora de que los clientes tengan dificultades o no para el pago. De las comentadas en el notebook 02 despues de graficar, he seleccionado las que me parecen que pueden tener más significancia a la hora de determinar la existencia o no de dificultades.

El tipo de contrato: 
pese a que la diferencia en proporción es muy pequeña, es notable que algunos clientes tienen más dificultades  si el tipo de préstamo es 'Cash Loan', esto, como ya comenté en los gráficos, puede deberse a la diferencia en cuanto a flexibilidad entre los 'Cash loans' y los 'Revolving loans'.

La edad:

La edad a priorí era ya de suponer que iba a afectar a tener dificultades en el pago, los usuarios más jovenes, con menos tiempo de vida para haber ahorrado y tener un 'colchón financiero' más amplio, tienen aparentemente mayores dificultades para afrontar los pagos.

Empleo desempeñado:

El empleo, y por lo tanto, la fuente de ingresos del cliente, es también una variable que puede explicar si el usuario tuvo dificultades de pago, los clientes con trabajos menos cualificados tienen mayores dificultades a la hora de realizar los pagos frente a aquellos que tienen un trabajo que requiere de mayor cualificación.

Otras variables:

Otras variables que a priorí solo funcionan si se cumplen los supuestos explicados en el notebook 02 son:
- EXT_SOURCE_1,2 y 3: de la cual tenemos poca información y es simplemente información externalizada.
- FLAG_DOCUMENT_2: algo similar, no sabemos la tipología ni la información que recoge el documento, pero parece afectar el no entregarlo de cara a tener dificultades.
- DAYS_REGISTRATION/ID_PUBLISH/LAST_PHONE_CHANGE: También hice una serie de suposiciones que podrían explicar el por qué aquellos con menos días desde el cambio de documentos, registro o teléfono tuvieron más dificultades a la hora de afrontar el pago